<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing_this_time_it_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, \
                             Dropout, Input, MaxPool2D, Rescaling, BatchNormalization
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incomplete53T1QB/cifar100-train.tfrecord*...:   0%|        …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incomplete53T1QB/cifar100-test.tfrecord*...:   0%|         …

Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


In [ ]:
ds_info.features

FeaturesDict({
    'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
    'id': Text(shape=(), dtype=string),
    'image': Image(shape=(32, 32, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
})

### Dataset to numpy:

In [3]:
# Create a subset of ds_train and ds_val
ds_train_subset = ds_train.take(500)
ds_val = ds_train.skip(600).take(50)

# Convert datasets to np arrays
def dataset_to_numpy(dataset):
    features = []
    labels = []

    for feature, label in tfds.as_numpy(dataset):
        features.append(feature)
        labels.append(label)

    return np.array(features), np.array(labels)

# Convert ds_train_subset and ds_val to np arrays
np_train_features, np_train_labels = dataset_to_numpy(ds_train_subset)
np_val_features, np_val_labels = dataset_to_numpy(ds_val)

### Preprocessing #2

In [4]:
def preprocess(image, label):
    image = tf.image.per_image_standardization(image)

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

In [5]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
ds_info.features['label'].num_classes

ds_train = ds_train.cache().shuffle(1000).batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTO)

ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [13]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_VGG

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_Resnet50

In [15]:
base_model_A = VGG16(include_top = False, weights = 'imagenet')
base_model_B = ResNet50(include_top = False, weights = 'imagenet')

base_model_A.trainable = False
base_model_B.trainable = False

In [ ]:
for layer in base_model_A.layers:
  layer